In [ ]:
# minimal example
# neo4j 3.x
# python neo4j-driver 1.7.x

In [ ]:
# neo4j-driver 1.7.6 api:https://neo4j.com/docs/api/python-driver/1.7/
# 参考【值得一看】：https://we-yun.com/doc/neo4j-doc/neo4j-pdf/neo4j-driver-manual-3.5-python.pdf

In [1]:
#!pip install neo4j-driver==1.7.6

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/183.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/183.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/183.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/183.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/183.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/183.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/183.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/183

In [6]:
from neo4j import GraphDatabase

# 简单示例

In [3]:
# 创建连接
url = "bolt://localhost:7687"
username = "neo4j"
password = "password"
# 高版本neo4j 会报错
# neo4j 4.1.0 测试通过
driver = GraphDatabase.driver(url, auth=(username, password), encrypted=False)

In [12]:
# 简单create测试
with driver.session() as session:
    session.run(
        """
create (a:etl {name:$name, age:$age})
""",
        age=30,
        name="Andy",
    )


In [20]:
# 简单match测试
with driver.session() as session:
    result = session.run(
        """MATCH (a:etl) RETURN a"""
    )
    for record in result:
        print(record["a"])

<Node id=0 labels={'etl'} properties={'name': 'Andy', 'age': 30}>


In [21]:
# Transaction functions
# 文档推荐的写法,可以分离我们的 数据查询 和 应用逻辑

# 首先封装我们的查询sql
def create_person_node(tx, name):
    return tx.run("""MATCH (a:etl{name:$name}) RETURN a"""
                  , name=name
                  ).single().value()
# 调用
with driver.session() as session:
    # Caller for transactional unit of work
    res=session.write_transaction(create_person_node, "Andy")
    print(res)

<Node id=0 labels={'etl'} properties={'name': 'Andy', 'age': 30}>


# 优雅的封装

In [9]:
class Neo4jHandler(object):
    """
    内部封装了逻辑，外部直接调用
    """

    def __init__(self, uri, user, password):
        # private driver
        self._driver = GraphDatabase.driver(uri, auth=(user, password), encrypted=False)

    def close(self):
        self._driver.close()

    # 执行自定义cql
    def run_cql(self, cql_str, **args):
        with self._driver.session() as session:
            return session.run(cql_str, **args)

    # 创建节点 create_etl_node+_create_etl_node
    def create_etl_node(self, **args):
        with self._driver.session() as session:
            return session.write_transaction(self._create_etl_node, **args)

    # 必须加staticmethod
    @staticmethod
    def _create_etl_node(tx, **args):
        return (
            tx.run(
                """ create (a:etl {name:$name}) RETURN a""",
                **args
            )
            .single()
            .value()
        )
    # def get_etl

# 示例

In [3]:
# [a,b] 表示 由 a 表生成 b 表
etl_table=[
    ["ENTENTPRBSCINFO","ENTENTPRBSCINFOHFS"],
    ["ENTENTPRBSCINFO","ENTENTPRBSCINFOTMP"],
    ["ENTENTPRINFOCHNG","ENTENTPRINFOCHNGTMP"],
    ["ENTINDSYCMRCANLRPTBSCINFO","ENTINDSYCMRCANLRPTBSCINFOHFS"],
    ["ENTINDSYCMRCANLRPTBSCINFO","ENTINDSYCMRCANLRPTBSCINFOFULI"],
    ["ENTINDSYCMRCCHTLCOLLINFO","ENTINDSYCMRCCHTLCOLLINFOHFS"],
    ["ENTINDSYCMRCCHTLCOLLINFO","ENTINDSYCMRCCHTLCOLLINFOTMP"],
    ["ENTINDSYCMRCENTPRMNSTFINFO","ENTINDSYCMRCENTPRMNSTFINFOHFS"],
    ["ENTINDSYCMRCENTPRMNSTFINFO","ENTINDSYCMRCENTPRMNSTFINFOTMP"],
    ["ENTINDSYCMRCENTPRSHRHR","ENTINDSYCMRCENTPRSHRHRHFS"],
    ["ENTINDSYCMRCENTPRSHRHR","ENTINDSYCMRCENTPRSHRHRTMP"],
    ["ENTINDSYCMRCEOTYPLDGBSCINFOTMP","ENTINDSYCMRCEOTYPLDGBSCINFOHFS"],
    ["ENTINDSYCMRCEOTYPLDGBSCINFO","ENTINDSYCMRCEOTYPLDGBSCINFOTMP"],
    ["ENTINDSYCMRCGRNTSCP","ENTINDSYCMRCGRNTSCPTMP"],
    ["ENTINDSYCMRCLGLASST","ENTINDSYCMRCLGLASSTSHFS"],
    ["ENTINDSYCMRCLGLASST","ENTINDSYCMRCLGLASSTTMP"],
    ["ENTINDSYCMRCMRTGCHNGINFO","ENTINDSYCMRCMRTGCHNGINFOHFS"],
    ["ENTINDSYCMRCMRTGCHNGINFO","ENTINDSYCMRCMRTGCHNGINFOTMP"],
    ["ENTINDSYCMRCMRTGRGSTRINFO",",ENTINDSYCMRCMRTGRGSTRINFOHFS"],
    ["ENTINDSYCMRCMRTGRGSTRINFO",",ENTINDSYCMRCMRTGRGSTRINFOTMP"],
    ["ENTINDSYCMRCMRTGGMRTGRINFOTMP",",ENTINDSYCMRCMRTGGMRTGRINFOHFS"],
    ["ENTINDSYCMRCMRTGGMRTGRINFO",",ENTINDSYCMRCMRTGGMRTGRINFOTMP"],
]

In [11]:
# 去重得到节点
etl_node=set(elem for inner_list in etl_table for elem in inner_list)
# 关系
etl_relation=etl_table

In [12]:
url = "bolt://localhost:7687"
username = "neo4j"
password = "password"
neo4j_handler=Neo4jHandler(url, username, password)

In [13]:
from superstream import Stream
(Stream(etl_node)
 .for_each(lambda x:neo4j_handler.create_etl_node(name=x)))

In [32]:
neo4j_handler.create_etl_node(name="Andy",age=30)

<Node id=12 labels={'etl'} properties={'name': 'Andy', 'age': 30}>

In [50]:
# 执行自定义 cql
neo4j_handler.run_cql(
    "CREATE (a:Person {name: $name})",
    name="Andy",
)